In [ ]:
%pip install -q matplotlib 
%pip install -q seaborn
%pip install -q pandas
%pip install -q numpy

In [2]:
import matplotlib.pyplot as plt 
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
import seaborn as sns
import pandas as pd
import numpy as np
import os 

In [18]:
ior_results_dirs = [
    'ior_results_n-1_seg-1_20250904',
    'ior_results_n-1_seg-6_20250904',
    'ior_results_n-2_seg-1_20250904'
]

# for each csv in each dir
# take the highest bandwidth row
df = pd.DataFrame()
for dir in ior_results_dirs:
    for file_name in os.listdir(dir):
        if file_name.endswith('.csv'): 
            file = os.path.join(dir, file_name)
            tmp = pd.read_csv(file)
            tmp = (tmp.iloc[tmp.idxmax(numeric_only=True)['bw(MiB/s)']].to_frame().T)
            tmp['fromCSV'] = file_name
            df = pd.concat([df,tmp], ignore_index=True)

df.to_csv('ior_results.csv')